In [1]:
%%help

Magic,Example,Explanation
info,%%info,Outputs session information for the current Livy endpoint.
cleanup,%%cleanup -f,"Deletes all sessions for the current Livy endpoint, including this notebook's session. The force flag is mandatory."
delete,%%delete -f -s 0,Deletes a session by number for the current Livy endpoint. Cannot delete this kernel's session.
logs,%%logs,Outputs the current session's Livy logs.
configure,"%%configure -f{""executorMemory"": ""1000M"", ""executorCores"": 4}",Configure the session creation parameters. The force flag is mandatory if a session has already been created and the session will be dropped and recreated.Look at Livy's POST /sessions Request Body for a list of valid parameters. Parameters must be passed in as a JSON string.
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
display,%%display -n 100my_spark_df,"Displays Spark dataframe as a beautiful HTML table with horizontal and vertical scroll bars(if necessary). Parameters: -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe to display. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
sql,%%sql -o tables -qSHOW TABLES,"Executes a SQL query against the variable sqlContext (Spark v1.x) or spark (Spark v2.x). Parameters: -o VAR_NAME: The result of the SQL query will be available in the %%local Python context as a Pandas dataframe. -q: The magic will return None instead of the dataframe (no visualization). -m, -n, -r are the same as the %%spark parameters above."
local,%%locala = 1,All the code in subsequent lines will be executed locally. Code must be valid Python code.


In [2]:
%%info

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *

ss = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1613257472994_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql.types import *

penschema = StructType([
    StructField("pix1",DoubleType(),True),
    StructField("pix2",DoubleType(),True),
    StructField("pix3",DoubleType(),True),
    StructField("pix4",DoubleType(),True),
    StructField("pix5",DoubleType(),True),
    StructField("pix6",DoubleType(),True),
    StructField("pix7",DoubleType(),True),
    StructField("pix8",DoubleType(),True),
    StructField("pix9",DoubleType(),True),
    StructField("pix10",DoubleType(),True),
    StructField("pix11",DoubleType(),True),
    StructField("pix12",DoubleType(),True),
    StructField("pix13",DoubleType(),True),
    StructField("pix14",DoubleType(),True),
    StructField("pix15",DoubleType(),True),
    StructField("pix16",DoubleType(),True),
    StructField("label",DoubleType(),True)
])

pen_raw = ss.read.csv("s3://usfca-msan694/penbased.dat", penschema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
pen_raw.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+
|pix1| pix2|pix3| pix4| pix5| pix6| pix7| pix8|pix9|pix10|pix11|pix12|pix13|pix14|pix15|pix16|label|
+----+-----+----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+
|47.0|100.0|27.0| 81.0| 57.0| 37.0| 26.0|  0.0| 0.0| 23.0| 56.0| 53.0|100.0| 90.0| 40.0| 98.0|  8.0|
| 0.0| 89.0|27.0|100.0| 42.0| 75.0| 29.0| 45.0|15.0| 15.0| 37.0|  0.0| 69.0|  2.0|100.0|  6.0|  2.0|
| 0.0| 57.0|31.0| 68.0| 72.0| 90.0|100.0|100.0|76.0| 75.0| 50.0| 51.0| 28.0| 25.0| 16.0|  0.0|  1.0|
| 0.0|100.0| 7.0| 92.0|  5.0| 68.0| 19.0| 45.0|86.0| 34.0|100.0| 45.0| 74.0| 23.0| 67.0|  0.0|  4.0|
| 0.0| 67.0|49.0| 83.0|100.0|100.0| 81.0| 80.0|60.0| 60.0| 40.0| 40.0| 33.0| 20.0| 47.0|  0.0|  1.0|
+----+-----+----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+
only showing top 5 rows

In [6]:
%%display -n 10
pen_raw

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [7]:
pen_raw.write.saveAsTable("pen")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Table `pen` already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 777, in saveAsTable
    self._jwrite.saveAsTable(name)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Table `pen` already exists.;'



In [8]:
%%sql
SELECT * FROM pen

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols = pen_raw.columns[:-1], outputCol="features")
penlpoints = va.transform(pen_raw).select("features", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
pendtsets = penlpoints.randomSplit([0.8, 0.2])
peddtrain = pendtsets[0].cache()
pentvalid = pendtsets[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=1000, regParam=0.3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
lrm = lr.fit(peddtrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
penpredict = lrm.transform(pentvalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator().setMetricName("f1")
evaluator.evaluate(penpredict)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8238576799491266

In [15]:
pen_raw.write.parquet("s3://usfca-msan694/pen_raw")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
ss.read.parquet("s3://usfca-msan694/pen_raw").printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- pix1: double (nullable = true)
 |-- pix2: double (nullable = true)
 |-- pix3: double (nullable = true)
 |-- pix4: double (nullable = true)
 |-- pix5: double (nullable = true)
 |-- pix6: double (nullable = true)
 |-- pix7: double (nullable = true)
 |-- pix8: double (nullable = true)
 |-- pix9: double (nullable = true)
 |-- pix10: double (nullable = true)
 |-- pix11: double (nullable = true)
 |-- pix12: double (nullable = true)
 |-- pix13: double (nullable = true)
 |-- pix14: double (nullable = true)
 |-- pix15: double (nullable = true)
 |-- pix16: double (nullable = true)
 |-- label: double (nullable = true)